# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import re
import glob
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
from IPython.display import display

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

[Why use Spark over Pandas](https://www.indellient.com/blog/a-journey-from-pandas-to-spark-data-frames/)

[1](https://github.com/the1schwartz/DEND-capstone/blob/master/City%20Immigration%20Demographics%20and%20Temperatures.ipynb)

[2](https://github.com/cheuklau/udacity-capstone/blob/master/Capstone%20Project%20Template.ipynb)

[3](https://github.com/nadirl00/Data-Engineering-Capstone-Project/blob/master/Capstone%20Project%20Template.ipynb)

In [11]:
spark = SparkSession.builder.\
    config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
    .enableHiveSupport()\
    .getOrCreate()

### Reading in immigration data

In [12]:
# read in the sample csv to pandas for easy assessing
pd_immig_df = pd.read_csv('immigration_data_sample.csv')

print('Finished reading immigration csv and sas.sample files')

Finished reading immigration csv and sas.sample files


### Reading in temperature data

In [13]:
#fname = '../../data2/GlobalLandTemperaturesByCity.csv'
#df_temp_spark = spark.read.csv(fname, sep = ',', header = 'true')

pd_temp_df = pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
print('Finished reading Temperature file to spark')

Finished reading Temperature file to spark


### The Data at a glance

In [14]:
pd.set_option('display.max_columns', 35)
pd_immig_df.head(10)

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,61.0,2.0,1.0,20160422,NaN,NaN,G,O,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,26.0,2.0,1.0,20160423,MTR,NaN,G,R,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,76.0,2.0,1.0,20160407,NaN,NaN,G,O,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,25.0,2.0,1.0,20160428,DOH,NaN,G,O,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,19.0,2.0,1.0,20160406,NaN,NaN,Z,K,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT
5,721257,1481650.0,2016.0,4.0,577.0,577.0,ATL,20552.0,1.0,GA,20606.0,51.0,2.0,1.0,20160408,NaN,NaN,T,N,NaN,M,1965.0,10072016,M,NaN,DL,7.368526e+08,910,B2
6,1072780,2197173.0,2016.0,4.0,245.0,245.0,SFR,20556.0,1.0,CA,20635.0,48.0,2.0,1.0,20160412,NaN,NaN,T,O,NaN,M,1968.0,10112016,F,NaN,CX,7.863122e+08,870,B2
7,112205,232708.0,2016.0,4.0,113.0,135.0,NYC,20546.0,1.0,NY,20554.0,33.0,2.0,1.0,20160402,NaN,NaN,G,O,NaN,M,1983.0,06302016,F,NaN,BA,5.547449e+10,00117,WT
8,2577162,5227851.0,2016.0,4.0,131.0,131.0,CHI,20572.0,1.0,IL,20575.0,39.0,2.0,1.0,20160428,NaN,NaN,O,O,NaN,M,1977.0,07262016,NaN,NaN,LX,5.941342e+10,00008,WT
9,10930,13213.0,2016.0,4.0,116.0,116.0,LOS,20545.0,1.0,CA,20553.0,35.0,2.0,1.0,20160401,NaN,NaN,O,O,NaN,M,1981.0,06292016,NaN,NaN,AA,5.544979e+10,00109,WT


The columns 'occup', 'entdepu', and 'insnum' is allways NaN, it looks like. 
Let's investigete this more later. 

In [15]:
# count non-NaN for each column
pd_immig_df.count()

Unnamed: 0    1000
cicid         1000
i94yr         1000
i94mon        1000
i94cit        1000
i94res        1000
i94port       1000
arrdate       1000
i94mode       1000
i94addr        941
depdate        951
i94bir        1000
i94visa       1000
count         1000
dtadfile      1000
visapost       382
occup            4
entdepa       1000
entdepd        954
entdepu          0
matflag        954
biryear       1000
dtaddto       1000
gender         859
insnum          35
airline        967
admnum        1000
fltno          992
visatype      1000
dtype: int64

In [16]:
# first look at the temperatur data
pd_temp_df.head(10)

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E
9,1744-08-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


Some rows look useless with AverageTemperature = NaN 

Reading in the immigrants data to Spark to take advantage of the parallelization of Spark

In [18]:
# full imigration file, only use with EMR cluster
#full_immi_fname = glob.glob("../../data/18-83510-I94-Data-2016/*.sas7bdat")

# sample immigration file
sample_immi_fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'

# read in the data to Spark
spark_immi_df =spark.read.format('com.github.saurfang.sas.spark').load(sample_immi_fname)
print('Finished reading immigration sas.sample files to Spark')

Finished reading immigration sas.sample files to Spark


In [19]:
spark_immi_df.limit(10).toPandas().head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,37.0,2.0,1.0,None,None,None,T,None,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,20130811,SEO,None,G,None,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,20160401,None,None,T,O,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,20160401,None,None,O,O,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,20160401,None,None,O,O,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [20]:
# count the number of rows
spark_immi_df.count()

3096313

This is a test to see if writing files work.

In [156]:
spark_immi_df.write.parquet("sas_data")
spark_immi_df = spark.read.parquet("sas_data")
print('Finished checking')

AnalysisException: 'path file:/home/workspace/sas_data already exists.;'

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Cleaning Immigration data

In [123]:
# Create list of valid ports
i94_sas_label_descriptions_fname = "I94_SAS_Labels_Descriptions.SAS"
with open(i94_sas_label_descriptions_fname) as f:
    lines = f.readlines()

re_compiled = re.compile(r"\'(.*)\'.*\'(.*)\'")
valid_ports = {}
for line in lines[302:961]:
    results = re_compiled.search(line)
    valid_ports[results[1]] = [results[2]]
print(len(valid_ports))


659


In [103]:
# count non-NaN for each column
#df_pd_immigration.count()

In [125]:

i94_immi_df.isnull().sum()
# Let's check for null values.
#df_temperature

AttributeError: 'DataFrame' object has no attribute 'isnull'

Looks like there are some columns that are full of NaN's, so dropping those. 

In [63]:
#df_pd_immigration_dropped = df_pd_immigration.drop(columns = ['occup', 'entdepu', 'insnum'])
#df_pd_immigration_dropped.count()

In [ ]:
df_temp_spark.limit(10).toPandas()

In [ ]:
for col_name in df.columns:
    print (col_name, end="---->")
    print (sum(df[col_name].isnull()))

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.